# 11.1. Output Formatting

In [1]:
repr(set('supercalifragilisticexpialidocious'))

"{'g', 's', 'e', 'd', 'r', 'c', 'a', 'l', 'i', 'f', 't', 'x', 'o', 'u', 'p'}"

In [2]:
import reprlib
reprlib.repr(set('supercalifragilisticexpialidocious'))

"{'a', 'c', 'd', 'e', 'f', 'g', ...}"

In [3]:
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta',
    'yellow'], 'blue']]]

pprint.pprint(t, width=30)

[[[['black', 'cyan'],
   'white',
   ['green', 'red']],
  [['magenta', 'yellow'],
   'blue']]]


In [4]:
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""

print(textwrap.fill(doc, width=40))

The wrap() method is just like fill()
except that it returns a list of strings
instead of one big string with newlines
to separate the wrapped lines.


In [5]:
import locale
locale.setlocale(locale.LC_ALL, 'English_United States.1252')

'English_United States.1252'

In [6]:
conv = locale.localeconv()          # get a mapping of conventions
x = 1234567.8
locale.format("%d", x, grouping=True)

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


'1,234,567'

In [7]:
locale.format_string("%s%.*f", (conv['currency_symbol'],
                     conv['frac_digits'], x), grouping=True)

'$1,234,567.80'

# 11.2. Templating

In [8]:
from string import Template
t = Template('${village}folk send $$10 to $cause.')
t.substitute(village='Nottingham', cause='the ditch fund')

'Nottinghamfolk send $10 to the ditch fund.'

In [9]:
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')
t.substitute(d)

KeyError: 'owner'

In [10]:
t.safe_substitute(d)

'Return the unladen swallow to $owner.'

In [11]:
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'
fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')

Enter rename style (%d-date %n-seqnum %f-format):  %n on %d, %f


In [12]:
t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))

img_1074.jpg --> 0 on 01Dec19, .jpg
img_1076.jpg --> 1 on 01Dec19, .jpg
img_1077.jpg --> 2 on 01Dec19, .jpg


# 11.3. Working with Binary Data Record Layouts

The struct module provides pack() and unpack() functions for working with variable length binary record formats. The following example shows how to loop through header information in a ZIP file without using the zipfile module. Pack codes "H" and "I" represent two and four byte unsigned numbers respectively. The "<" indicates that they are standard size and in little-endian byte order:



In [15]:
import struct

with open('fibo.zip', 'rb') as f:
    data = f.read()

start = 0
for i in range(3):                      # show the first 3 file headers
    start += 14
    fields = struct.unpack('<IIIHH', data[start:start+16])
    crc32, comp_size, uncomp_size, filenamesize, extra_size = fields

    start += 16
    filename = data[start:start+filenamesize]
    start += filenamesize
    extra = data[start:start+extra_size]
    print(filename, hex(crc32), comp_size, uncomp_size)

    start += extra_size + comp_size     # skip to the next header

b'fibo.py' 0x756baaf7 162 347
b'' 0xaaf74f79 10646891 22740992


error: unpack requires a buffer of 16 bytes

# 11.4. Multi-threading

In [16]:
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile

    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of:', self.infile)

background = AsyncZip('fibo.py', 'fibo.zip')
background.start()
print('The main program continues to run in foreground.')

background.join()    # Wait for the background task to finish
print('Main program waited until background was done.')

The main program continues to run in foreground.
Finished background zip of: fibo.py
Main program waited until background was done.


While those tools are powerful, minor design errors can result in problems that are difficult to reproduce. So, the preferred approach to task coordination is to concentrate all access to a resource in a single thread and then use the queue module to feed that thread with requests from other threads. Applications using Queue objects for inter-thread communication and coordination are easier to design, more readable, and more reliable.



# 11.5. Logging

In [17]:
import logging
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found', 'server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

ERROR:root:Error occurred
CRITICAL:root:Critical error -- shutting down


By default, informational and debugging messages are suppressed and the output is sent to standard error. Other output options include routing messages through email, datagrams, sockets, or to an HTTP Server. New filters can select different routing based on message priority: DEBUG, INFO, WARNING, ERROR, and CRITICAL.



# 11.6. Weak References

Python does automatic memory management (reference counting for most objects and garbage collection to eliminate cycles). The memory is freed shortly after the last reference to it has been eliminated.

This approach works fine for most applications but occasionally there is a need to track objects only as long as they are being used by something else. Unfortunately, just tracking them creates a reference that makes them permanent. The weakref module provides tools for tracking objects without creating a reference. When the object is no longer needed, it is automatically removed from a weakref table and a callback is triggered for weakref objects. Typical applications include caching objects that are expensive to create:

In [18]:
import weakref, gc
class A:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)

In [19]:
a = A(10)                   # create a reference
d = weakref.WeakValueDictionary()
d['primary'] = a            # does not create a reference
d['primary']                # fetch the object if it is still alive

10

In [20]:
del a                       # remove the one reference
gc.collect()                # run garbage collection right away

3218

In [22]:
d['primary']                # entry was automatically removed

10

The code above should have caused KeyError: 'primary' on the last statement.

# 11.7. Tools for Working with Lists

The array module provides an array() object that is like a list that stores only homogeneous data and stores it more compactly. The following example shows an array of numbers stored as two byte unsigned binary numbers (typecode "H") rather than the usual 16 bytes per entry for regular lists of Python int objects:



In [24]:
from array import array
a = array('H', [4000, 10, 700, 22222])
sum(a)

26932

In [25]:
a[1:3]

array('H', [10, 700])

The collections module provides a deque() object that is like a list with faster appends and pops from the left side but slower lookups in the middle. These objects are well suited for implementing queues and breadth first tree searches:



In [26]:
from collections import deque
d = deque(["task1", "task2", "task3"])
d.append("task4")
print("Handling", d.popleft())

Handling task1


In [27]:
unsearched = deque([starting_node])
def breadth_first_search(unsearched):
    node = unsearched.popleft()
    for m in gen_moves(node):
        if is_goal(m):
            return m
        unsearched.append(m)

NameError: name 'starting_node' is not defined

In addition to alternative list implementations, the library also offers other tools such as the bisect module with functions for manipulating sorted lists:



In [29]:
import bisect
scores = [(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]
bisect.insort(scores, (300, 'ruby'))
scores


[(100, 'perl'), (200, 'tcl'), (300, 'ruby'), (400, 'lua'), (500, 'python')]

The heapq module provides functions for implementing heaps based on regular lists. The lowest valued entry is always kept at position zero. This is useful for applications which repeatedly access the smallest element but do not want to run a full list sort:



In [30]:
from heapq import heapify, heappop, heappush
data = [1, 3, 5, 7, 9, 2, 4, 6, 8, 0]
heapify(data)                      # rearrange the list into heap order
heappush(data, -5)                 # add a new entry
[heappop(data) for i in range(3)]  # fetch the three smallest entries


[-5, 0, 1]

In [31]:
data

[2, 3, 4, 6, 9, 5, 8, 7]

# 11.8. Decimal Floating Point Arithmetic


The decimal module offers a Decimal datatype for decimal floating point arithmetic. Compared to the built-in float implementation of binary floating point, the class is especially helpful for:
- financial applications and other uses which require exact decimal representation,
- control over precision,
- control over rounding to meet legal or regulatory requirements,
- tracking of significant decimal places, or
- applications where the user expects the results to match calculations done by hand.

In [33]:
from decimal import *
round(Decimal('0.70') * Decimal('1.05'), 2)

Decimal('0.74')

In [32]:
round(.70 * 1.05, 2)

0.73

Decimal reproduces mathematics as done by hand and avoids issues that can arise when binary floating point cannot exactly represent decimal quantities. Exact representation enables the Decimal class to perform modulo calculations and equality tests that are unsuitable for binary floating point:

In [35]:
Decimal('1.00') % Decimal('.10')

Decimal('0.00')

In [36]:
1.00 % 0.10

0.09999999999999995

In [37]:
sum([Decimal('0.1')]*10) == Decimal('1.0')

True

In [38]:
sum([0.1]*10) == 1.0

False

In [39]:
getcontext().prec = 36
Decimal(1) / Decimal(7)

Decimal('0.142857142857142857142857142857142857')